In [1]:
# used packages
import pandas as pd
import os
import datetime
import random
from typing import List, Union
import itertools
import time

In [2]:
# load data
df_meter_list = pd.read_excel(os.path.join(os.getcwd(), 'gorilla_test_data.xlsx'), sheet_name = 'meter_list') 
df_forecast = pd.read_excel(os.path.join(os.getcwd(), 'gorilla_test_data.xlsx'), sheet_name = 'forecast_table')
df_rate = pd.read_excel(os.path.join(os.getcwd(), 'gorilla_test_data.xlsx'), sheet_name = 'rate_table')

# Question 1


In [3]:
#merge the meters list and forecast table
merged_df = df_meter_list.merge(df_forecast,on='meter_id')

In [4]:
def calculate_daily_cost(row: pd.Series) -> pd.Series:
    """
    Calculate the daily cost by multiplying the rate (in p/kWh) with the day's forecast (in kWh)
    Arguments:
        row: pd.Series
    Returns:
        The row with 2 additional columns "daily_charges" and "daily_cost" which represent the daily rate and the cost in Pounds for the day
    """
    date_max = df_rate[(df_rate['exit_zone'] == row.exit_zone) & (df_rate['date'] < row.date)].date.max()
    df_filtered_on_zone_date = df_rate[(df_rate['exit_zone'] == row.exit_zone) & (df_rate['date'] == date_max)]
    rate_p_per_kwh = df_filtered_on_zone_date[(row.aq_kwh > df_filtered_on_zone_date.aq_min_kwh) & ((row.aq_kwh < df_filtered_on_zone_date.aq_max_kwh) | (df_filtered_on_zone_date.aq_max_kwh.isnull()))].rate_p_per_kwh
    row['daily_charges'] = rate_p_per_kwh.iloc[0]
    row['daily_cost'] = (row['kwh'] * row['daily_charges']) * 0.01
    
    return row

In [5]:
def calculate_cost_table(meters_table: pd.DataFrame, forecast_table: pd.DataFrame, rate_table: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the total estimated consumption and total cost per meter ID over the entire forecast period
    Arguments:
        meters_table: pd.DataFrame
        forecast_table: pd.DataFrame
        rate_table: pd.DataFrame
    Returns:
        The dataframe with the total estimated consumption (kWh) and total cost in Pounds per meter ID
    """
    merged_df = meters_table.merge(forecast_table,on='meter_id')
    
    # calculate table with the daily cost
    calculated_df = merged_df.apply(lambda x: calculate_daily_cost(x), axis = 1)
    calculated_df.kwh = calculated_df.kwh.round(2)
    
    # calculate summary statistics
    result_df = calculated_df.groupby(['meter_id'])[['kwh', 'daily_cost']].sum().round(2)
    
    # change column names
    result_df.index.name = "Meter ID"
    result_df = result_df.rename(columns={"kwh": "Total Estimated Consumption (kWh)", "daily_cost": "Total Cost (£)"})

    return result_df

In [6]:
#calculate cost table
final_df = calculate_cost_table(df_meter_list, df_forecast, df_rate)

In [7]:
# the resulting table
final_df

,Total Estimated Consumption (kWh),Total Cost (£)
Meter ID,,
14676236,28978.04,100.07
34509937,78323.80,275.28
50264822,265667.04,730.63
88357331,484399.00,1432.13


# Question 2

In [8]:
exit_zone_list = df_rate.exit_zone.unique()

In [9]:
print('the maximum forecasted kwh: {}'.format(df_forecast.kwh.max()))
print('the minimum forecasted kwh: {}'.format(df_forecast.kwh.min()))

the maximum forecasted kwh: 1169.400411138627
the minimum forecasted kwh: 2.91877038164736


In [10]:
print('the maximum aq kwh: {}'.format(df_meter_list.aq_kwh.max()))
print('the minimum aq kwh: {}'.format(df_meter_list.aq_kwh.min()))

the maximum aq kwh: 484399
the minimum aq kwh: 28978


In [11]:
def generate_random_meters_list(number: int = 1) -> pd.DataFrame:
    """
    Randomly generate meter list data
    Arguments:
        number: int
    Returns:
        The dataframe with the randomly generated meter list data
    """
    random_meter_list: List[List[Union[int, str]]] = []
    for ind in range(0, number):
        exit_zone: str = random.choice(exit_zone_list)
        meter: int = random.randint(10000000, 99999999)
        aq_kwh: int = random.randint(1000, 1000000)
        meter: List[Union[int, str]] = [meter, exit_zone, aq_kwh]
        random_meter_list.append(meter)
    
    df_random_meter: pd.DataFrame = pd.DataFrame(random_meter_list, columns = ['meter_id', 'exit_zone' , 'aq_kwh'])
    return df_random_meter    

In [12]:
result = generate_random_meters_list(5)

In [13]:
result

,meter_id,exit_zone,aq_kwh
0,72044836,WA2,453765
1,10395065,SW1,341866
2,68705463,LW,698813
3,36001497,WM3,206987
4,11525096,SW3,631182


# Question 3

In [14]:
date = datetime.datetime(2021, 5, 17)

In [15]:
def generate_forecast_consumption_table(meter_list: List[int], date: datetime, duration: int = 1) -> pd.DataFrame:
    """
    Calculate the total estimated consumption and total cost per meter ID over the entire forecast period
    Arguments:
        meters_table: pd.DataFrame
        forecast_table: pd.DataFrame
        rate_table: pd.DataFrame
    Returns:
        The dataframe with the total estimated consumption (kWh) and total cost in Pounds per meter ID
    """
    df_list = []
    for meter in meter_list: 
        df: pd.DataFrame = pd.DataFrame({'meter_id': meter, 'date':pd.date_range(date, periods=duration)})
        df['kwh']: pd.DataFrame = df.apply(lambda x: random.uniform(0, 1000), axis = 1)
        df_list.append(df)
    
    final_df = pd.concat(df_list)
    return final_df

In [16]:
generate_forecast_consumption_table([154340, 954090], date, 70)

,meter_id,date,kwh
0,154340,2021-05-17,784.397450
1,154340,2021-05-18,749.354287
2,154340,2021-05-19,615.107814
3,154340,2021-05-20,5.380821
4,154340,2021-05-21,890.156368
...,...,...,...
65,954090,2021-07-21,381.730097
66,954090,2021-07-22,704.704922
67,954090,2021-07-23,46.557253
68,954090,2021-07-24,811.239748


# Question 4

In [17]:
meters_list = [1, 5, 10, 50, 100]
duration = [1, 10, 100, 500, 1000]
combinations = list(itertools.product(meters_list, duration))

In [18]:
combinations

[(1, 1),
 (1, 10),
 (1, 100),
 (1, 500),
 (1, 1000),
 (5, 1),
 (5, 10),
 (5, 100),
 (5, 500),
 (5, 1000),
 (10, 1),
 (10, 10),
 (10, 100),
 (10, 500),
 (10, 1000),
 (50, 1),
 (50, 10),
 (50, 100),
 (50, 500),
 (50, 1000),
 (100, 1),
 (100, 10),
 (100, 100),
 (100, 500),
 (100, 1000)]

In [19]:
date = datetime.datetime(2020, 6, 20)
for combination in combinations:
    number_of_meters = combination[0]
    duration_of_forecast = combination[1]
    
    meters_df = generate_random_meters_list(number_of_meters)
    forecasted_consumption_table = generate_forecast_consumption_table(list(meters_df.meter_id.values), date, duration_of_forecast)
    
    start = time.perf_counter()
    calculate_cost_table(meters_df, forecasted_consumption_table, df_rate)
    print(f"Completed the execution for {number_of_meters} number of meters and {duration_of_forecast} forecast periods in {round(time.perf_counter() - start, 2)} seconds")
    

Completed the execution for 1 number of meters and 1 forecast periods in 0.04 seconds
Completed the execution for 1 number of meters and 10 forecast periods in 0.06 seconds
Completed the execution for 1 number of meters and 100 forecast periods in 0.24 seconds
Completed the execution for 1 number of meters and 500 forecast periods in 1.25 seconds
Completed the execution for 1 number of meters and 1000 forecast periods in 3.01 seconds
Completed the execution for 5 number of meters and 1 forecast periods in 0.01 seconds
Completed the execution for 5 number of meters and 10 forecast periods in 0.11 seconds
Completed the execution for 5 number of meters and 100 forecast periods in 1.71 seconds
Completed the execution for 5 number of meters and 500 forecast periods in 5.45 seconds
Completed the execution for 5 number of meters and 1000 forecast periods in 11.61 seconds
Completed the execution for 10 number of meters and 1 forecast periods in 0.03 seconds
Completed the execution for 10 numbe

# Question 5

The function does not scale with an increasing number of forecast periods or/and more meters. The solution for this problem would be to use a parrallel processing framework like Spark.